<img src="https://upload.wikimedia.org/wikipedia/fr/8/81/Sciences_SU.png" width="240" height="240" align="center"/>


# MLA 801. Auto-Encodeur (en Keras, avec CNN)

# Partie 1 : AE-CNN sur données propres

## 1.1 Importation des modules

In [ ]:
# On importe les librairies nécessaires

# Le dataset MNIST
from tensorflow.keras.datasets import mnist

# Les librairies TF pour le DL
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense, Conv2D, Conv2DTranspose, MaxPooling2D, Flatten, UpSampling2D, Reshape
from tensorflow.keras.models import Model,Sequential

# Les librairies habituelles
import numpy as np
import matplotlib.pyplot as plt

# On configure la session pour l'utilisation de GPU
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#session = tf.Session(config=config)

# On désactive les warnings
import warnings
warnings.filterwarnings('ignore')

## 1.2 Définition de fonctions nécessaires

In [ ]:
def MNIST_AE_disp(img_in, img_out, img_idx):

    num_img = len(img_idx)
    plt.figure(figsize=(18, 4))

    for i, image_idx in enumerate(img_idx):
        # on trace l'image originale
        ax = plt.subplot(2, num_img, i + 1)
        plt.imshow(img_in[image_idx].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # on trace l'image bruitée
        ax = plt.subplot(2, num_img, num_img + i + 1)
        plt.imshow(img_out[image_idx].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()


## 1.3 Chargement et formatage des données (propres)

In [ ]:
# On charge les données de MNIST (incluses dans Keras)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# On normalise les valeurs des pixels de [0, 255] à [0, 1]
x_train=x_train.astype('float32')/float(x_train.max())
x_test=x_test.astype('float32')/float(x_test.max())

# POUR LES CNN : On rajoute une dimension pour spécifier qu'il s'agit d'imgages en NdG
x_train=x_train.reshape(len(x_train),x_train.shape[1], x_train.shape[2], 1)
x_test=x_test.reshape(len(x_test),x_test.shape[1], x_test.shape[2], 1)

# On inspecte les dimensions de nos données
# Base d'entrainement : 60,000 images de dimension (28,28)
# Base de test : 10,000 images de dimension (28,28)
print("Training set : ",x_train.shape)
print("Testing set : ",x_test.shape)

## 1.4 Création du réseau CNN

In [ ]:
# On déclare les dimensions de notre réseau
 
############################################
# 1) Construction de la partie "encodeur"  #
############################################

autoencoder = Sequential()

# 1) Partie encodeur
autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=x_train.shape[1:]))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))

# 2) Construction de la partie "décodeur"
# Decoder Layers
autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
#autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
autoencoder.add(Conv2D(1, (3, 3), activation='relu', padding='same'))

# On visualise la structure du réseau
autoencoder.summary()

# Compilation du réseau
autoencoder.compile(loss='binary_crossentropy', optimizer='adam')
 

## 1.5 Apprentissage du réseau

In [ ]:
# Apprentissage du réseau
history=autoencoder.fit(x_train,x_train,epochs=5, batch_size=256, shuffle=True, validation_data=(x_test,x_test))

## 1.6 Prédiction par le réseau

In [ ]:
# On génère un jeu de 10 images test au hasard
num_images=10
np.random.seed(42)
random_test_images=np.random.randint(x_test.shape[0], size=num_images)

# On détermine l'image encodée et l'image décodée
decoded_img=autoencoder.predict(x_test)

## 1.7 Visualisation

In [ ]:
# 1) On visualise l'image d'entrée, le code, et la reconstruction

# On trace l'image d'entrée et l'image décodée (reconstruite à partir du code)
MNIST_AE_disp(x_test, decoded_img, random_test_images)


# Partie 2 : Vers le Denoising AE

## 2.1 Création des données bruitées

In [ ]:
# On génère un jeu de 10 images test au hasard
num_images=10
np.random.seed(42)
random_test_images=np.random.randint(x_test.shape[0], size=num_images)

# On génère des données bruitées

# 1) Possibilité 1: Ajout de bruit additif
x_train_noisy = x_train + np.random.normal(loc=0.0, scale=0.5, size=x_train.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)

x_test_noisy = x_test + np.random.normal(loc=0.0, scale=0.5, size=x_test.shape)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# 2) Possibilité 2 : ajout de bruit multiplicatif avec un masque binaire
#dropout_frac = 0.15
# On créée des masques binaires avec une certaine probabilité de 1
#activation_train_mask = np.random.binomial(1, dropout_frac, size=x_train.shape)
#activation_test_mask = np.random.binomial(1, dropout_frac, size=x_test.shape)
# On applique les masques (multiplication point à point)
#x_train_noisy = np.multiply(x_train, activation_train_mask)
#x_test_noisy = np.multiply(x_test, activation_test_mask)

# On visualise les données

MNIST_AE_disp(x_train, x_train_noisy, random_test_images)


## 2.2 Prédictions à partir des données bruitées

In [ ]:
# Même chose mais avec les données bruitées

num_images = 10
np.random.seed(42)
random_test_images = np.random.randint(x_test.shape[0], size=num_images)

# Denoise test images
x_test_denoised = autoencoder.predict(x_test_noisy)

# On visualise les données
MNIST_AE_disp(x_test_noisy, x_test_denoised, random_test_images)


## 2.3 Apprentissage du réseaux DAE

In [ ]:
# On ré-apprend le réseau mais en mode denoiser ...

# Apprentissage du réseau
history=autoencoder.fit(x_train_noisy,x_train,epochs=5, batch_size=256, shuffle=True, validation_data=(x_test_noisy,x_test))

## 2.4 Prédiction et visualisation à partir des données bruitées

In [ ]:
# On génère un jeu de 10 images test au hasard
num_images=10
np.random.seed(42)
random_test_images=np.random.randint(x_test.shape[0], size=num_images)

# On détermine l'image encodée et l'image décodée
decoded_img=autoencoder.predict(x_test)

# Même chose mais avec les données bruitées
MNIST_AE_disp(x_test_noisy, decoded_img, random_test_images)